<a href="https://colab.research.google.com/github/rosehelfrich/Ky_School_data/blob/main/additional/File_2_KY_AST_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

df = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/All Years/df_header.csv')

df_header = df.loc[:,['District Code', 'District', 'School Code', 'School', 'Level']]

# Load reports

In [5]:
profile_2012 = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/2011-2012/ACCOUNTABILITY_PROFILE-2012.csv') # total points available
profile_2013 = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/2012-2013/ACCOUNTABILITY_PROFILE-2013.csv') # total points available
profile_2014 = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/2013-2014/ACCOUNTABILITY_PROFILE-2014.csv')
profile_2015 = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/2014-2015/ACCOUNTABILITY_PROFILE-2015.csv')
profile_2016 = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/2015-2016/ACCOUNTABILITY_PROFILE-2016.csv')

profile_2017 = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/2016-2017/ACCOUNTABILITY_ACHIEVEMENT_SUMMARY.csv') # 'Overall Score' are all null
profile_2018 = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/2017-2018/ACCOUNTABILITY_SUMMARY-2018.csv')
profile_2019 = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/2018-2019/ACCOUNTABILITY_PROFILE-2019.csv')

profile_2022 = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/2021-2022/accountability_profile_2022.csv')

In [6]:
profile_2017 = profile_2017[~profile_2017['SCH_NAME'].str.contains("---")]
profile_2017 = profile_2017[profile_2017['PERFORMANCE_TYPE'] == 'Points']
profile_2017 = profile_2017[profile_2017['ACCT_TYPE'] == 'ACH']
profile_2017.reset_index(drop=True, inplace=True)

In [7]:
profile_2019 = profile_2019[~profile_2019['SCH_NAME'].str.contains("---")]
profile_2019.reset_index(drop=True, inplace=True)

In [8]:
profile_2022.dropna(subset='SCHOOL TYPE', axis =0, inplace=True)
profile_2022.reset_index(drop=True, inplace=True)

In [9]:
score_2012 = profile_2012.loc[:,['SCH_YEAR', 'SCH_CD', 'OVERALL_SCORE', 'CLASSIFICATION', 'CONTENT_LEVEL']]
score_2013 = profile_2013.loc[:,['SCH_YEAR', 'SCH_CD', 'OVERALL_SCORE', 'CLASSIFICATION', 'CONTENT_LEVEL']]
score_2014 = profile_2014.loc[:,['SCH_YEAR', 'SCH_CD', 'OVERALL_SCORE', 'CLASSIFICATION', 'CONTENT_LEVEL']]
score_2015 = profile_2015.loc[:,['SCH_YEAR', 'SCH_CD', 'OVERALL_SCORE', 'CLASSIFICATION', 'CONTENT_LEVEL']]
score_2016 = profile_2016.loc[:,['SCH_YEAR', 'SCH_CD', 'OVERALL_SCORE', 'CLASSIFICATION', 'CONTENT_LEVEL']]

score_2017 = profile_2017.loc[:,['SCH_YEAR', 'SCH_CD', 'TOTAL_POINTS', 'CONTENT_LEVEL']]
score_2018 = profile_2018.loc[:,['SCH_YEAR', 'SCH_CD', 'PROFICIENCY_RATE', 'LEVEL']] # All Null values for Proficiency Rating

score_2019 = profile_2019.loc[:,['SCH_YEAR', 'SCH_CD', 'PROFICIENCY_RATE', 'PROFICIENCY_RATING', 'LEVEL']] # PROFICIENCY rate available
score_2022 = profile_2022.loc[:,['SCHOOL YEAR','SCHOOL CODE', 'OVERALL INDICATOR RATE', 'OVERALL INDICATOR RATING', 'LEVEL']]  # PROFICIENCY scores NOT available


In [10]:
# Rename columns

def four_columns_renamed(df, end_year):
  df.columns = ['End Year', 'School Code', 'Proficiency Rate', 'Classification', 'Level']
  df.replace(df['End Year'][0], end_year, inplace=True)

four_columns_renamed(score_2012, 2012)
four_columns_renamed(score_2013, 2013)
four_columns_renamed(score_2014, 2014)
four_columns_renamed(score_2015, 2015)
four_columns_renamed(score_2016, 2016)
four_columns_renamed(score_2019, 2019)
four_columns_renamed(score_2022, 2022)

def three_columns_renamed(df, end_year):
  df.columns = ['End Year', 'School Code', 'Proficiency Rate', 'Level']
  df.replace(df['End Year'][0], end_year, inplace=True)

three_columns_renamed(score_2017, 2017)
three_columns_renamed(score_2018, 2018)


# Concat and remove duplicates

In [11]:
all_scores = pd.concat([score_2012, score_2013, score_2014, score_2015, score_2016,
                        score_2017, score_2018, score_2019, score_2022], axis =0)
all_scores.reset_index(drop=True, inplace=True)

In [12]:
# Clean up Classification values
all_scores['Level'].replace(['Elementary School', 'Middle School', 'High School'], ['ES', 'MS', 'HS'], inplace=True)
all_scores['Classification'].replace([1,2,3,4,5], [0,1,2,3,4], inplace=True)
all_scores['Classification'].replace(['Needs Improvement/Progressing', 'Proficient/Progressing', 'Distinguished/Progressing'],
                                     ['Needs Improvement', 'Proficient', 'Distinguished'], inplace=True)
all_scores['Classification'].replace(['Very Low', 'Low', 'Medium', 'High', 'Very High'],
                     [0,1,2,3,4], inplace=True)

In [13]:
# Create a new column 'Rating' in df_scores
all_scores['Rating Code'] = all_scores['Classification']
all_scores['Rating Code'].replace(['Needs Improvement', 'Proficient', 'Distinguished'], np.NaN, inplace=True)
all_scores['Classification'].replace([0,1,2,3,4], np.NaN, inplace=True)


# Merge with header

In [14]:
df_scores = pd.merge(df_header, all_scores, on=['School Code', 'Level'], how='inner')
df_scores.sort_values(by=['End Year', 'District', 'School'], ascending=True, inplace=True)
df_scores.reset_index(drop=True, inplace=True)

In [15]:
# Reorder columns
reordered_columns = ['End Year', 'District Code', 'District', 'School Code', 'School', 'Level',
                     'Proficiency Rate', 'Classification', 'Rating Code']
df_scores = df_scores[reordered_columns]

In [16]:
df_scores.to_csv('/content/drive/MyDrive/Colab Data/KY EPSB/All Years/Unpublished/df_scores.csv', index = False)